In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
!pip install -q scann

In [3]:
from typing import Dict, Text

import os
import pprint
import tempfile

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
import tensorflow_recommenders as tfrs

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from pathlib import Path
import pandas as pd
import numpy as np
import random
from IPython.display import Image
from IPython.core.display import HTML 
import pdb;

from subprocess import check_output
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# Importing Libraries and cookbooks
from recsys import *## recommender system cookbook
#from generic_preprocessing import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
from itertools import combinations, chain, product

In [11]:
demo = pd.read_csv("demo001.csv",sep="|")

demo.head(10)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID_CTE,FECHA_NAC,GENERO,EDO_CIVIL,D_EDO,DESC_D_EDO,D_NC,DESC_D_NC,TP_HOGAR,T_HOGAR,...,FECHADIFCOBRO,SDOCTAPERDIDA,FECHACORTE,TIPOTELEFONO,NUMEROTELEFONOORIGEN,NUMEROTELEFONO,CARRIER,DES_CORREOELECTRONICO,DES_ORIGEN,SAMPLING
0,2746824,1952-09-22 00:00:00.000000,F,C,9.0,AGUASCALIENTES,36.0,AGUASCALIENTES,P,1988-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,1.0,9.130889e+06,4.499131e+09,0.0,CONCHITAGUILAR2@GMAIL.COM,Coppel.com en tienda,0.004464
1,6572723,1958-11-04 00:00:00.000000,F,C,10.0,JALISCO,67.0,TONALA,P,1992-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,4.0,6.170833e+06,3.336171e+09,0.0,MOGLEZZB1102@HOTMAIL.COM,Tienda,0.005916
2,12334731,1980-03-14 00:00:00.000000,M,C,23.0,VERACRUZ,40.0,CORDOBA,R,2012-01-01 00:00:00.000000,...,2011-08-31 00:00:00.000000,0.0,2020-07-31 00:00:00.000000,1.0,2.711111e+09,2.711111e+09,0.0,JORGECONTRERAS_08@HOTMAIL.COM,Tienda,0.003026
3,12408172,1968-02-19 00:00:00.000000,M,S,18.0,PUEBLA,85.0,ATLIXCO,P,2007-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,1.0,0.000000e+00,2.444463e+09,0.0,MARIO_1018@HOTMAIL.COM,Coppel.com,0.002311
4,24428035,1950-03-18 00:00:00.000000,M,D,25.0,DISTRITO FEDERAL,107.0,IZTAPALAPA,F,2003-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,3.0,5.541973e+09,5.541973e+09,0.0,BARBIE_COMEX@HOTMAIL.COM,CAT,0.008750
5,42979358,1963-10-12 00:00:00.000000,M,S,15.0,ZACATECAS,273.0,SOMBRERETE,P,1978-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,4.331055e+09,4.331055e+09,39.0,B.LINDA.MORALES@GMAIL.COM,Tienda,0.007626
6,49727061,1970-02-17 00:00:00.000000,F,C,4.0,BAJA CALIFORNIA NORTE,64.0,ENSENADA,P,2010-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,5.562271e+09,5.562271e+09,39.0,LOLISMAGA_24@HOTMAIL.COM,Tienda,0.006580
7,26835752,1984-06-14 00:00:00.000000,F,C,23.0,VERACRUZ,183.0,TUXPAN,P,2013-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,7.681044e+09,7.681044e+09,39.0,NaN,NaN,0.007558
8,25066445,1974-10-12 00:00:00.000000,F,C,22.0,HIDALGO,316.0,MINERAL DE LA R,P,2019-12-09 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,2.0,5.520714e+09,5.520714e+09,39.0,NaN,NaN,0.009830
9,46495652,1986-11-02 00:00:00.000000,M,S,31.0,OAXACA,384.0,LOMA BONITA,P,2004-01-01 00:00:00.000000,...,NaN,0.0,2020-07-31 00:00:00.000000,5.0,2.871169e+09,2.871169e+09,0.0,NaN,NaN,0.002984


In [12]:
from datetime import datetime
from datetime import date

In [13]:
demo["Fecha_Nac"]=pd.to_datetime(demo["FECHA_NAC"], format='%Y-%m-%d %H:%M')
demo["Fecha_Nac"].head(5)

0   1952-09-22
1   1958-11-04
2   1980-03-14
3   1968-02-19
4   1950-03-18
Name: Fecha_Nac, dtype: datetime64[ns]

In [14]:
hoy=datetime.now().year
def age(fecnan):
    return (hoy-fecnan.year)

demo["Edad"]=demo["Fecha_Nac"].apply(age)
demo["Edad"].head(5)

0    69.0
1    63.0
2    41.0
3    53.0
4    71.0
Name: Edad, dtype: float64

In [15]:
demo["edad"]=pd.cut(demo['Edad'], [0,25,35,45,55,65,90]).astype(str)
demo.head()

,ID_CTE,FECHA_NAC,GENERO,EDO_CIVIL,D_EDO,DESC_D_EDO,D_NC,DESC_D_NC,TP_HOGAR,T_HOGAR,...,TIPOTELEFONO,NUMEROTELEFONOORIGEN,NUMEROTELEFONO,CARRIER,DES_CORREOELECTRONICO,DES_ORIGEN,SAMPLING,Fecha_Nac,Edad,edad
0,2746824,1952-09-22 00:00:00.000000,F,C,9.0,AGUASCALIENTES,36.0,AGUASCALIENTES,P,1988-01-01 00:00:00.000000,...,1.0,9.130889e+06,4.499131e+09,0.0,CONCHITAGUILAR2@GMAIL.COM,Coppel.com en tienda,0.004464,1952-09-22,69.0,"(65.0, 90.0]"
1,6572723,1958-11-04 00:00:00.000000,F,C,10.0,JALISCO,67.0,TONALA,P,1992-01-01 00:00:00.000000,...,4.0,6.170833e+06,3.336171e+09,0.0,MOGLEZZB1102@HOTMAIL.COM,Tienda,0.005916,1958-11-04,63.0,"(55.0, 65.0]"
2,12334731,1980-03-14 00:00:00.000000,M,C,23.0,VERACRUZ,40.0,CORDOBA,R,2012-01-01 00:00:00.000000,...,1.0,2.711111e+09,2.711111e+09,0.0,JORGECONTRERAS_08@HOTMAIL.COM,Tienda,0.003026,1980-03-14,41.0,"(35.0, 45.0]"
3,12408172,1968-02-19 00:00:00.000000,M,S,18.0,PUEBLA,85.0,ATLIXCO,P,2007-01-01 00:00:00.000000,...,1.0,0.000000e+00,2.444463e+09,0.0,MARIO_1018@HOTMAIL.COM,Coppel.com,0.002311,1968-02-19,53.0,"(45.0, 55.0]"
4,24428035,1950-03-18 00:00:00.000000,M,D,25.0,DISTRITO FEDERAL,107.0,IZTAPALAPA,F,2003-01-01 00:00:00.000000,...,3.0,5.541973e+09,5.541973e+09,0.0,BARBIE_COMEX@HOTMAIL.COM,CAT,0.008750,1950-03-18,71.0,"(65.0, 90.0]"


In [16]:
demo["NDepen"]=demo['N_D_EC'].apply(lambda x: 0 if x == 0 else 1).astype(str)
demo.head()

,ID_CTE,FECHA_NAC,GENERO,EDO_CIVIL,D_EDO,DESC_D_EDO,D_NC,DESC_D_NC,TP_HOGAR,T_HOGAR,...,NUMEROTELEFONOORIGEN,NUMEROTELEFONO,CARRIER,DES_CORREOELECTRONICO,DES_ORIGEN,SAMPLING,Fecha_Nac,Edad,edad,NDepen
0,2746824,1952-09-22 00:00:00.000000,F,C,9.0,AGUASCALIENTES,36.0,AGUASCALIENTES,P,1988-01-01 00:00:00.000000,...,9.130889e+06,4.499131e+09,0.0,CONCHITAGUILAR2@GMAIL.COM,Coppel.com en tienda,0.004464,1952-09-22,69.0,"(65.0, 90.0]",0
1,6572723,1958-11-04 00:00:00.000000,F,C,10.0,JALISCO,67.0,TONALA,P,1992-01-01 00:00:00.000000,...,6.170833e+06,3.336171e+09,0.0,MOGLEZZB1102@HOTMAIL.COM,Tienda,0.005916,1958-11-04,63.0,"(55.0, 65.0]",1
2,12334731,1980-03-14 00:00:00.000000,M,C,23.0,VERACRUZ,40.0,CORDOBA,R,2012-01-01 00:00:00.000000,...,2.711111e+09,2.711111e+09,0.0,JORGECONTRERAS_08@HOTMAIL.COM,Tienda,0.003026,1980-03-14,41.0,"(35.0, 45.0]",1
3,12408172,1968-02-19 00:00:00.000000,M,S,18.0,PUEBLA,85.0,ATLIXCO,P,2007-01-01 00:00:00.000000,...,0.000000e+00,2.444463e+09,0.0,MARIO_1018@HOTMAIL.COM,Coppel.com,0.002311,1968-02-19,53.0,"(45.0, 55.0]",0
4,24428035,1950-03-18 00:00:00.000000,M,D,25.0,DISTRITO FEDERAL,107.0,IZTAPALAPA,F,2003-01-01 00:00:00.000000,...,5.541973e+09,5.541973e+09,0.0,BARBIE_COMEX@HOTMAIL.COM,CAT,0.008750,1950-03-18,71.0,"(65.0, 90.0]",0


In [17]:
df_demo=demo.loc[:,('ID_CTE', "GENERO", 'EDO_CIVIL',"edad","NDepen","TP_HOGAR")]
df_demo["user_id"]=df_demo["ID_CTE"].astype(str)
df_demo.head(10)


,ID_CTE,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR,user_id
0,2746824,F,C,"(65.0, 90.0]",0,P,2746824
1,6572723,F,C,"(55.0, 65.0]",1,P,6572723
2,12334731,M,C,"(35.0, 45.0]",1,R,12334731
3,12408172,M,S,"(45.0, 55.0]",0,P,12408172
4,24428035,M,D,"(65.0, 90.0]",0,F,24428035
5,42979358,M,S,"(55.0, 65.0]",1,P,42979358
6,49727061,F,C,"(45.0, 55.0]",1,P,49727061
7,26835752,F,C,"(35.0, 45.0]",0,P,26835752
8,25066445,F,C,"(45.0, 55.0]",0,P,25066445
9,46495652,M,S,"(25.0, 35.0]",1,P,46495652


In [18]:
tran = pd.read_csv("transactional001.csv",sep="|")

tran.head(10)

,ID_CTE,FECHA_TICKET,ID_TIENDA,ID_TICKET,PROD_SKU,PROD_DEP,DESC_PROD_DEP,PROD_CLAS,DESC_PROD_CLAS,PROD_FAM,DESC_PROD_FAM,PROD_AREA,PROD_CATEGORIA,DESC_PROD_CATEGORIA,IMPORTE_VTA,CANTIDAD_VTA,PROD_PRECIO_PROMEDIO,TIPOCOMPRA,FECHACORTE
0,145035,2017-10-02 00:00:00.000000,3,133548,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,3448.00,1,3448.00,Credito,2017-10-31 00:00:00.000000
1,145035,2018-05-08 00:00:00.000000,1252,7400,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,5646.00,1,5646.00,Credito,2018-05-31 00:00:00.000000
2,145035,2019-08-23 00:00:00.000000,3,48708,215063,2,ELECTRONICA,24,TELEFONOS CELULARES,85,AT.T,Muebles,1,Celulares,2154.32,1,2154.32,Credito,2019-08-31 00:00:00.000000
3,145035,2019-09-20 00:00:00.000000,5,23998,800404,8,ZAPATERIA,60,ZAPATO DAMA T 22.5-27,48,SENORA CONFORT,Ropa,17,Zapato Dama,603.00,1,603.00,Credito,2019-09-30 00:00:00.000000
4,145035,2020-01-13 00:00:00.000000,2,171365,-99,-99,Sin Descripcion,-99,Sin Descripcion,-99,Sin Descripcion,Prestamos,-99,Sin Descripcion,17241.00,1,17241.00,Credito,2020-01-31 00:00:00.000000
5,558561,2017-03-17 00:00:00.000000,119,505516,340634,3,MUEBLE SUELTO,30,JARDIN,11,MESA PLEGABLE,Muebles,3,Hogar,1119.83,1,1119.83,Credito,2017-03-31 00:00:00.000000
6,558561,2017-10-17 00:00:00.000000,119,526127,236785,2,ELECTRONICA,29,CHIPS CELULARES,11,CHIP 4G MULTIFORMATO,Muebles,1,Celulares,0.00,1,0.00,Contado,2017-10-31 00:00:00.000000
7,558561,2017-10-17 00:00:00.000000,119,526127,236905,2,ELECTRONICA,95,ACCESORIOS CELULARES,2,MICAS,Muebles,1,Celulares,128.45,1,128.45,Credito,2017-10-31 00:00:00.000000
8,558561,2017-10-17 00:00:00.000000,119,526127,253311,2,ELECTRONICA,24,TELEFONOS CELULARES,9,TELCEL R2,Muebles,1,Celulares,2671.55,1,2671.55,Credito,2017-10-31 00:00:00.000000
9,558561,2017-10-17 00:00:00.000000,119,526127,296475,2,ELECTRONICA,95,ACCESORIOS CELULARES,27,PROTECTOR DE PANTALLA VIDRIO,Muebles,1,Celulares,171.55,1,171.55,Credito,2017-10-31 00:00:00.000000


In [19]:
df_tran=tran[((tran["PROD_AREA"]=="Muebles")  | (tran["PROD_AREA"]=="Ropa"))&((tran["IMPORTE_VTA"]>100))]


In [20]:
df_tran["area"]=df_tran["PROD_AREA"].apply(lambda x: 1 if x == 'Ropa' else 2) 
df_tran["fam"]=df_tran["area"].astype(str)+df_tran["PROD_DEP"].astype(str)+df_tran["PROD_CLAS"].astype(str).str.zfill(2)+df_tran["PROD_FAM"].astype(str).str.zfill(3)
df_tran["clas"]=df_tran["area"].astype(str)+df_tran["PROD_DEP"].astype(str)+df_tran["PROD_CLAS"].astype(str).str.zfill(2)
df_tran["idclas"]=df_tran["clas"]
df_tran["idfam"]=df_tran["fam"]
df_tran["movie_id"]=df_tran["fam"].astype(str)#df_tran["DESC_PROD_DEP"]#
df_tran["user_id"]=df_tran["ID_CTE"].astype(str)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [21]:
df_f=df_tran.loc[:,('user_id', 'FECHA_TICKET', 'movie_id')].groupby(['user_id',"movie_id"]).size().reset_index()
df_f.head(10)

,user_id,movie_id,0
0,10000040,1105068,1
1,10000040,1204616,1
2,10000040,1701216,1
3,10000040,1863254,2
4,10000212,1106055,3
5,10000212,1867308,2
6,10000212,1869214,1
7,10000212,2224057,1
8,10000292,1101650,1
9,10000292,1312150,1


In [22]:
df_f1=df_f[:50000]

In [23]:
dff1=pd.merge(df_f1, df_demo, on='user_id', how='inner')

dff1.head(10)

,user_id,movie_id,0,ID_CTE,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR
0,10000040,1105068,1,10000040,M,S,"(35.0, 45.0]",0,P
1,10000040,1204616,1,10000040,M,S,"(35.0, 45.0]",0,P
2,10000040,1701216,1,10000040,M,S,"(35.0, 45.0]",0,P
3,10000040,1863254,2,10000040,M,S,"(35.0, 45.0]",0,P
4,10000212,1106055,3,10000212,M,S,"(35.0, 45.0]",0,P
5,10000212,1867308,2,10000212,M,S,"(35.0, 45.0]",0,P
6,10000212,1869214,1,10000212,M,S,"(35.0, 45.0]",0,P
7,10000212,2224057,1,10000212,M,S,"(35.0, 45.0]",0,P
8,10000292,1101650,1,10000292,M,U,"(45.0, 55.0]",1,P
9,10000292,1312150,1,10000292,M,U,"(45.0, 55.0]",1,P


In [24]:
df_1=dff1.dropna() 
df_1.head(10)

,user_id,movie_id,0,ID_CTE,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR
0,10000040,1105068,1,10000040,M,S,"(35.0, 45.0]",0,P
1,10000040,1204616,1,10000040,M,S,"(35.0, 45.0]",0,P
2,10000040,1701216,1,10000040,M,S,"(35.0, 45.0]",0,P
3,10000040,1863254,2,10000040,M,S,"(35.0, 45.0]",0,P
4,10000212,1106055,3,10000212,M,S,"(35.0, 45.0]",0,P
5,10000212,1867308,2,10000212,M,S,"(35.0, 45.0]",0,P
6,10000212,1869214,1,10000212,M,S,"(35.0, 45.0]",0,P
7,10000212,2224057,1,10000212,M,S,"(35.0, 45.0]",0,P
8,10000292,1101650,1,10000292,M,U,"(45.0, 55.0]",1,P
9,10000292,1312150,1,10000292,M,U,"(45.0, 55.0]",1,P


In [25]:
dff=pd.DataFrame(df_1["movie_id"].unique())
dff["movie_id"]=dff[0].astype(str)
dff.head()

,0,movie_id
0,1105068,1105068
1,1204616,1204616
2,1701216,1701216
3,1863254,1863254
4,1106055,1106055


In [26]:
prod = pd.read_csv("prod_variables.csv",sep=",")

prod.head(10)

,id_Familia,Departamento,Clase,Familia,Categoria,Subcategoria
0,1101002,CABALLEROS,ACCESORIOS,NOVEDADES,Caballeros Accesorios,Caballeros Accesorios
1,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios
2,1101004,CABALLEROS,ACCESORIOS,GORRA,Caballeros Accesorios,Caballeros Accesorios
3,1101027,CABALLEROS,ACCESORIOS,CINTO MODA,Caballeros Accesorios,Caballeros Accesorios
4,1101028,CABALLEROS,ACCESORIOS,CINTO VESTIR,Caballeros Accesorios,Caballeros Accesorios
5,1101063,CABALLEROS,ACCESORIOS,PAÑUELO,Caballeros Accesorios,Caballeros Accesorios
6,1101083,CABALLEROS,ACCESORIOS,PLUMAS,Caballeros Accesorios,Caballeros Accesorios
7,1101342,CABALLEROS,ACCESORIOS,MESSENGER,Caballeros Accesorios,Caballeros Accesorios
8,1101649,CABALLEROS,ACCESORIOS,LENTES SOLARES,Caballeros Accesorios,Caballeros Accesorios
9,1101650,CABALLEROS,ACCESORIOS,RELOJES,Caballeros Accesorios,Caballeros Accesorios


In [27]:
prod["movie_id"]=prod["id_Familia"].astype(str)
prod["id_Familia"]=prod["id_Familia"].astype(str)
prod.head()

,id_Familia,Departamento,Clase,Familia,Categoria,Subcategoria,movie_id
0,1101002,CABALLEROS,ACCESORIOS,NOVEDADES,Caballeros Accesorios,Caballeros Accesorios,1101002
1,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios,1101003
2,1101004,CABALLEROS,ACCESORIOS,GORRA,Caballeros Accesorios,Caballeros Accesorios,1101004
3,1101027,CABALLEROS,ACCESORIOS,CINTO MODA,Caballeros Accesorios,Caballeros Accesorios,1101027
4,1101028,CABALLEROS,ACCESORIOS,CINTO VESTIR,Caballeros Accesorios,Caballeros Accesorios,1101028


In [28]:
dfff=df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfff.head(5)

,user_id,movie_id,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR
0,10000040,1105068,M,S,"(35.0, 45.0]",0,P
1,10000040,1204616,M,S,"(35.0, 45.0]",0,P
2,10000040,1701216,M,S,"(35.0, 45.0]",0,P
3,10000040,1863254,M,S,"(35.0, 45.0]",0,P
4,10000212,1106055,M,S,"(35.0, 45.0]",0,P


In [29]:
dfp=pd.merge(dfff, prod, on='movie_id', how='inner')

dfp.head(10)

,user_id,movie_id,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR,id_Familia,Departamento,Clase,Familia,Categoria,Subcategoria
0,10000040,1105068,M,S,"(35.0, 45.0]",0,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
1,10000520,1105068,F,C,"(55.0, 65.0]",1,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
2,10001185,1105068,M,S,"(35.0, 45.0]",0,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
3,10004981,1105068,M,C,"(55.0, 65.0]",1,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
4,10009338,1105068,F,C,"(55.0, 65.0]",1,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
5,10009900,1105068,M,C,"(55.0, 65.0]",1,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
6,10016033,1105068,M,C,"(35.0, 45.0]",1,F,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
7,1001809,1105068,F,C,"(65.0, 90.0]",0,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
8,10028459,1105068,F,C,"(45.0, 55.0]",1,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior
9,10029235,1105068,F,V,"(65.0, 90.0]",0,P,1105068,CABALLEROS,PLAYERA,MODA MC CR,Caballero Exterior,Caballero Exterior


In [30]:
dfpp=dff["movie_id"].unique()#pd.DataFrame({"movie_id" : dff.loc[:,("movie_id")]})#dff["movie_id"].unique()#dfp.loc[:,"movie_id"]
dfpp

array(['1105068', '1204616', '1701216', ..., '2296010', '2509015',
       '1542038'], dtype=object)

In [31]:
dfff=df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfff.head(5)

,user_id,movie_id,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR
0,10000040,1105068,M,S,"(35.0, 45.0]",0,P
1,10000040,1204616,M,S,"(35.0, 45.0]",0,P
2,10000040,1701216,M,S,"(35.0, 45.0]",0,P
3,10000040,1863254,M,S,"(35.0, 45.0]",0,P
4,10000212,1106055,M,S,"(35.0, 45.0]",0,P


In [32]:
dfp=pd.merge(prod,dfff, on='movie_id', how='inner')#df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfp.head(5)

,id_Familia,Departamento,Clase,Familia,Categoria,Subcategoria,movie_id,user_id,GENERO,EDO_CIVIL,edad,NDepen,TP_HOGAR
0,1101002,CABALLEROS,ACCESORIOS,NOVEDADES,Caballeros Accesorios,Caballeros Accesorios,1101002,10020036,M,U,"(35.0, 45.0]",1,F
1,1101002,CABALLEROS,ACCESORIOS,NOVEDADES,Caballeros Accesorios,Caballeros Accesorios,1101002,10635891,M,C,"(35.0, 45.0]",1,P
2,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios,1101003,10016033,M,C,"(35.0, 45.0]",1,F
3,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios,1101003,10059362,F,S,"(35.0, 45.0]",0,P
4,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios,1101003,1007701,F,C,"(45.0, 55.0]",0,P


In [33]:
dfmo=pd.DataFrame({"movie_id" : dff.loc[:,("movie_id")]})#dff["movie_id"].unique()#dfp.loc[:,"movie_id"]
dfmovie=pd.merge(prod,dfmo, on='movie_id', how='inner')
dfmovie.head()

,id_Familia,Departamento,Clase,Familia,Categoria,Subcategoria,movie_id
0,1101002,CABALLEROS,ACCESORIOS,NOVEDADES,Caballeros Accesorios,Caballeros Accesorios,1101002
1,1101003,CABALLEROS,ACCESORIOS,BILLETERA,Caballeros Accesorios,Caballeros Accesorios,1101003
2,1101004,CABALLEROS,ACCESORIOS,GORRA,Caballeros Accesorios,Caballeros Accesorios,1101004
3,1101027,CABALLEROS,ACCESORIOS,CINTO MODA,Caballeros Accesorios,Caballeros Accesorios,1101027
4,1101028,CABALLEROS,ACCESORIOS,CINTO VESTIR,Caballeros Accesorios,Caballeros Accesorios,1101028


In [36]:
ratings = tf.data.Dataset.from_tensor_slices((dfp.to_dict('list')))

In [38]:
ratings.take(10)

<TakeDataset shapes: {id_Familia: (), Departamento: (), Clase: (), Familia: (), Categoria: (), Subcategoria: (), movie_id: (), user_id: (), GENERO: (), EDO_CIVIL: (), edad: (), NDepen: (), TP_HOGAR: ()}, types: {id_Familia: tf.string, Departamento: tf.string, Clase: tf.string, Familia: tf.string, Categoria: tf.string, Subcategoria: tf.string, movie_id: tf.string, user_id: tf.string, GENERO: tf.string, EDO_CIVIL: tf.string, edad: tf.string, NDepen: tf.string, TP_HOGAR: tf.string}>

In [39]:
ratings

<TensorSliceDataset shapes: {id_Familia: (), Departamento: (), Clase: (), Familia: (), Categoria: (), Subcategoria: (), movie_id: (), user_id: (), GENERO: (), EDO_CIVIL: (), edad: (), NDepen: (), TP_HOGAR: ()}, types: {id_Familia: tf.string, Departamento: tf.string, Clase: tf.string, Familia: tf.string, Categoria: tf.string, Subcategoria: tf.string, movie_id: tf.string, user_id: tf.string, GENERO: tf.string, EDO_CIVIL: tf.string, edad: tf.string, NDepen: tf.string, TP_HOGAR: tf.string}>

In [40]:
dfr = tfds.as_dataframe(ratings.take(10))
dfr.head()

,Categoria,Clase,Departamento,EDO_CIVIL,Familia,GENERO,NDepen,Subcategoria,TP_HOGAR,edad,id_Familia,movie_id,user_id
0,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'U',b'NOVEDADES',b'M',b'1',b'Caballeros Accesorios',b'F',"b'(35.0, 45.0]'",b'1101002',b'1101002',b'10020036'
1,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'C',b'NOVEDADES',b'M',b'1',b'Caballeros Accesorios',b'P',"b'(35.0, 45.0]'",b'1101002',b'1101002',b'10635891'
2,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'C',b'BILLETERA',b'M',b'1',b'Caballeros Accesorios',b'F',"b'(35.0, 45.0]'",b'1101003',b'1101003',b'10016033'
3,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'S',b'BILLETERA',b'F',b'0',b'Caballeros Accesorios',b'P',"b'(35.0, 45.0]'",b'1101003',b'1101003',b'10059362'
4,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'C',b'BILLETERA',b'F',b'0',b'Caballeros Accesorios',b'P',"b'(45.0, 55.0]'",b'1101003',b'1101003',b'1007701'


In [42]:
movies= tf.data.Dataset.from_tensor_slices((dfmovie.to_dict('list')))

In [43]:
movies

<TensorSliceDataset shapes: {id_Familia: (), Departamento: (), Clase: (), Familia: (), Categoria: (), Subcategoria: (), movie_id: ()}, types: {id_Familia: tf.string, Departamento: tf.string, Clase: tf.string, Familia: tf.string, Categoria: tf.string, Subcategoria: tf.string, movie_id: tf.string}>

In [44]:
dfm = tfds.as_dataframe(movies.take(10))
dfm.head()

,Categoria,Clase,Departamento,Familia,Subcategoria,id_Familia,movie_id
0,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'NOVEDADES',b'Caballeros Accesorios',b'1101002',b'1101002'
1,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'BILLETERA',b'Caballeros Accesorios',b'1101003',b'1101003'
2,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'GORRA',b'Caballeros Accesorios',b'1101004',b'1101004'
3,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'CINTO MODA',b'Caballeros Accesorios',b'1101027',b'1101027'
4,b'Caballeros Accesorios',b'ACCESORIOS',b'CABALLEROS',b'CINTO VESTIR',b'Caballeros Accesorios',b'1101028',b'1101028'


In [45]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
#y=ratings.map(lambda x: x["user_id"])

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies.map(lambda x: x["movie_id"]))

In [46]:
#timestamps = np.concatenate(list(ratings.map(lambda x: x["GENERO"].astype(int)).batch(50)))

#max_timestamp = timestamps.max()
#min_timestamp = timestamps.min()

#timestamp_buckets = np.linspace(
 #   min_timestamp, max_timestamp, num=100,
#)

timestamps= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["GENERO"]))))

EC= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["EDO_CIVIL"]))))

ED= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["edad"]))))

DE= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["NDepen"]))))


TP= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["TP_HOGAR"]))))

cat= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["Categoria"]))))

fam= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["Familia"]))))


unique_movie_titles = np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["movie_id"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["user_id"]))))

In [47]:
class UserModel(tf.keras.Model):
  
  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=timestamps, mask_token=None),
      tf.keras.layers.Embedding(len(timestamps) + 1, 32)
        ])
    
      self.EC_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=EC, mask_token=None),
      tf.keras.layers.Embedding(len(EC) + 1, 32)
        ])
        
      self.ED_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=ED, mask_token=None),
      tf.keras.layers.Embedding(len(ED) + 1, 32)
        ])
    
      self.DE_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=DE, mask_token=None),
      tf.keras.layers.Embedding(len(DE) + 1, 32)
        ])
        
      self.TP_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=TP, mask_token=None),
      tf.keras.layers.Embedding(len(TP) + 1, 32)
       ])

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])
    #tf.print(inputs["user_id"])
    return tf.concat([
            self.user_embedding(inputs["user_id"]),
            self.timestamp_embedding(inputs["GENERO"]),
            self.EC_embedding(inputs["EDO_CIVIL"]),
            self.ED_embedding(inputs["edad"]),
            self.DE_embedding(inputs["NDepen"]),
            self.TP_embedding(inputs["TP_HOGAR"])
            ], axis=1)

In [48]:
class MovieModel(tf.keras.Model):
  
  def __init__(self,use_timestamps):
    super().__init__()
    self._use_timestamps = use_timestamps
    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])
    
    self.title_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    #self.title_vectorizer.adapt(movies)
    self.title_vectorizer.adapt(ratings.map(lambda x: x["movie_id"]))
    if use_timestamps:
        self.cat_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=cat, mask_token=None),
        tf.keras.layers.Embedding(len(cat) + 1, 32)
        ])
        
        self.fam_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=fam, mask_token=None),
        tf.keras.layers.Embedding(len(fam) + 1, 32)
        ])
    
   # def call(self, inputs):
    #if not self._use_timestamps:
     # return self.title_embedding(titles["movie_id"]),
      #  self.title_text_embedding(titles["movie_id"])
    
  def call(self, titles):
    if not self._use_timestamps:
      return tf.concat([
            self.title_embedding(titles["movie_id"]),
            self.title_text_embedding(titles["movie_id"]),
        ], axis=1)

      #tf.print(titles["movie_id"])
    return tf.concat([
            self.title_embedding(titles["movie_id"]),
            self.title_text_embedding(titles["movie_id"]),
            self.cat_embedding(titles["Categoria"]),
            self.fam_embedding(titles["Familia"])
            ], axis=1)

In [49]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "GENERO": features["GENERO"],
        "EDO_CIVIL": features["EDO_CIVIL"],
        "edad": features["edad"],
         "NDepen": features["NDepen"],
         "TP_HOGAR": features["TP_HOGAR"]
                                            })
    #movie_embeddings = self.candidate_model(features["movie_id"])
    
    movie_embeddings = self.candidate_model({
       "movie_id": features["movie_id"],
       "Categoria": features["Categoria"],
        "Familia": features["Familia"]
                                                 
                                                 })
    
    #movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [50]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(1_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(8_000)
test = shuffled.skip(8_000).take(2_000)

cached_train = train.shuffle(1_000).batch(204)
cached_test = test.batch(406).cache()

In [51]:
model = MovielensModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Consider rewriting this model with the Functional API.
Epoch 1/3
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
40/40 [==============================] - 8s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0182 - factorized_top_k/top_5_categorical_accuracy: 0.0820 - factorized_top_k/top_10_categorical_accuracy: 0.1484 - factorized_top_k/top_50_categorical_accuracy: 0.3088 - factorized_top_k/top_100_categorical_accuracy: 0.3478 - loss: 1040.8782 - regularization_loss: 0.0000e+00 - total_loss: 1040.8782
Epoch 2/3
40/40 [==============================] - 4s 109ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0298 - factorized_top_k/top_5_categorical_accuracy: 0.1265 - factorized_top_k/top_10_categorical_accuracy: 0.2120 - factorized_top_k/top_50_categorical_accuracy: 0.5052 - factorized_top_k/top_100_

In [52]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"accuracy (train): {train_accuracy:.2f}.")
print(f"accuracy (test): {test_accuracy:.2f}.")

Consider rewriting this model with the Functional API.
Epoch 1/3
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
40/40 [==============================] - 6s 107ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0235 - factorized_top_k/top_5_categorical_accuracy: 0.0699 - factorized_top_k/top_10_categorical_accuracy: 0.1135 - factorized_top_k/top_50_categorical_accuracy: 0.3380 - factorized_top_k/top_100_categorical_accuracy: 0.4644 - loss: 1205.4792 - regularization_loss: 0.0000e+00 - total_loss: 1205.4792
Epoch 2/3
40/40 [==============================] - 4s 108ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0051 - factorized_top_k/top_5_categorical_accuracy: 0.0564 - factorized_top_k/top_10_categorical_accuracy: 0.1065 - factorized_top_k/top_50_categorical_accuracy: 0.2860 - factorized_top_k/top_100_

In [53]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 127ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0025 - factorized_top_k/top_5_categorical_accuracy: 0.0190 - factorized_top_k/top_10_categorical_accuracy: 0.0420 - factorized_top_k/top_50_categorical_accuracy: 0.1525 - factorized_top_k/top_100_categorical_accuracy: 0.2685 - loss: 2457.4288 - regularization_loss: 0.0000e+00 - total_loss: 2457.4288


{'factorized_top_k/top_1_categorical_accuracy': 0.0024999999441206455,
 'factorized_top_k/top_5_categorical_accuracy': 0.01899999938905239,
 'factorized_top_k/top_10_categorical_accuracy': 0.041999999433755875,
 'factorized_top_k/top_50_categorical_accuracy': 0.1525000035762787,
 'factorized_top_k/top_100_categorical_accuracy': 0.2685000002384186,
 'loss': 2283.7548828125,
 'regularization_loss': 0,
 'total_loss': 2283.7548828125}

In [55]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
# recommends movies out of the entire movies dataset.
index.index(movies.map(lambda x: x["movie_id"]).batch(100).map(model.candidate_model), movies.map(lambda x: x["movie_id"]))

# Get recommendations.
#_, titles = index(tf.constant(["42"]))
#print(f"Recommendations for user 42: {titles[0, :3]}")

Cause: could not parse the source code of <function <lambda> at 0x7fdcc86e4d40>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: x['movie_id'])

Match 1:
(lambda x: x['movie_id'])

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fdcc86e4d40>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names.
Match 0:
(lambda x: x['movie_id'])

Match 1:
(lambda x: x['movie_id'])

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


TypeError: in user code:

    <ipython-input-48-bb187e6bd5a1>:55 call  *
        self.fam_embedding(titles["Familia"])
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:1009 _slice_helper
        _check_index(s)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:886 _check_index
        raise TypeError(_SLICE_TYPE_ERROR + ", got {!r}".format(idx))

    TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'movie_id'


In [54]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.query_model)
scann_index.index(movies.batch(100).map(model.candidate_model), movies)

Consider rewriting this model with the Functional API.


ValueError: Attempt to convert a value ({'id_Familia': <tf.Tensor: shape=(), dtype=string, numpy=b'1101002'>, 'Departamento': <tf.Tensor: shape=(), dtype=string, numpy=b'CABALLEROS'>, 'Clase': <tf.Tensor: shape=(), dtype=string, numpy=b'ACCESORIOS'>, 'Familia': <tf.Tensor: shape=(), dtype=string, numpy=b'NOVEDADES'>, 'Categoria': <tf.Tensor: shape=(), dtype=string, numpy=b'Caballeros Accesorios'>, 'Subcategoria': <tf.Tensor: shape=(), dtype=string, numpy=b'Caballeros Accesorios'>, 'movie_id': <tf.Tensor: shape=(), dtype=string, numpy=b'1101002'>}) with an unsupported type (<class 'dict'>) to a Tensor.

In [ ]:
# Get recommendations.
_, titles = scann_index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

In [73]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id","GENERO","EDO_CIVIL"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [74]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 1000)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,    
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 1000)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(28).map(movie_model)
  )
)

In [56]:
model.user_model.predict(np.array(["138"]))

array([[ 4.97999042e-03, -1.83736570e-02,  1.16316229e-03,
         2.74277665e-02,  4.98649962e-02,  4.56113927e-02,
         7.81090185e-03, -9.27485526e-04,  1.48377754e-02,
        -2.76878960e-02,  3.50721367e-02,  2.78701074e-02,
         4.71521057e-02,  4.84329574e-02,  2.44679302e-03,
         1.24264248e-02, -4.98382822e-02,  2.06773169e-02,
         2.30567716e-02, -1.29004568e-03, -2.83110868e-02,
         3.36977728e-02,  8.31341743e-03,  4.01573889e-02,
         3.33807133e-02, -3.51386182e-02, -1.10668167e-02,
        -4.34323922e-02,  4.95199226e-02,  2.52286531e-02,
        -3.01531442e-02,  4.68717851e-02, -3.16448361e-02,
         1.54444240e-02,  2.77019478e-02,  4.88602854e-02,
        -3.28195579e-02,  3.50382589e-02, -1.02340691e-02,
         4.31673639e-02,  3.62020992e-02, -2.88588293e-02,
        -7.40443543e-03, -1.72408670e-03, -2.38767266e-02,
        -3.37214619e-02, -3.79501358e-02, -9.16055590e-03,
        -4.91043106e-02, -4.01937738e-02,  9.68147069e-0